In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import time
from sklearn.ensemble import BaggingClassifier

In [2]:
variable = pd.read_csv("feature_selection_final.csv")
flabel = pd.read_csv("flabel.csv")
X_oot = pd.read_csv("data_final_OOT.csv")
# oot = flabel.iloc[:833507]["fraud_label"]

In [3]:
y = flabel.iloc[:833507]["fraud_label"]
y_oot = flabel.iloc[833507:]["fraud_label"]

In [4]:
X_oot.shape[0] == y_oot.shape[0]

True

In [5]:
vlist = list(variable.columns)
vlist.remove('record')

In [6]:
X= variable[vlist]
y = y
X_oot = X_oot[vlist]

In [7]:
X.shape[1] == X_oot.shape[1]

True

In [8]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [9]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
# x_train2,x_test2,y_train2,y_test2 = train_test_split(X,y,test_size=0.2)
# x_train3,x_test3,y_train3,y_test3 = train_test_split(X,y,test_size=0.2)

# model = [(x_train1,x_test1,y_train1,y_test1),(x_train2,x_test2,y_train2,y_test2),(x_train3,x_test3,y_train3,y_test3)]

In [10]:
x_train=scaler.fit_transform(x_train.to_numpy()) # x_train1 is a numpy array
x_train=pd.DataFrame(x_train,columns=X.columns)

In [11]:
x_test=scaler.fit_transform(x_test.to_numpy())
x_test=pd.DataFrame(x_test,columns=X.columns)

In [12]:
X_oot=scaler.fit_transform(X_oot.to_numpy())
X_oot=pd.DataFrame(X_oot,columns=X.columns)

In [13]:
def FDR(x,y,model):
    pred = model.predict_proba(x)[:,1]
    y_df = pd.DataFrame(y)
    y_df['pred'] = pred
    top = int(len(y_df) * 0.03)
    numbads = sum(y_df['fraud_label'] == 1)
    fdr = y_df.sort_values(by = 'pred', ascending = False).head(top).fraud_label.sum()/numbads
    return fdr

In [14]:
kernal = []
C=[]
gamma=[]
train = []
test = []
ott = []

In [15]:
n_estimators = 10
start = time.time()

clf = BaggingClassifier(svm.SVC(kernel='linear', probability=True,C=0.1), max_samples=1.0 / n_estimators, n_estimators=n_estimators)
clf.fit(x_train, y_train)
end = time.time()


[]

In [16]:
C.append(0.1)
gamma.append("-")
train.append(FDR(x_train,y_train,clf))
test.append(FDR(x_test,y_test,clf))
ott.append(FDR(X_oot,y_oot,clf))
print("Bagging SVC",0.1, end - start)

Bagging SVC 0.1 742.1075990200043


[0.48700754400670576]

In [17]:
# linear 
Cs = [1, 10]
for c in Cs:
    n_estimators = 10
    start = time.time()
    clf = BaggingClassifier(svm.SVC(kernel='linear', probability=True,C=c), max_samples=1.0 / n_estimators, n_estimators=n_estimators)
    clf.fit(x_train, y_train)
    end = time.time()
    kernal.append("linear")
    C.append(c)
    gamma.append("-")
    train.append(FDR(x_train,y_train,clf))
    test.append(FDR(x_test,y_test,clf))
    ott.append(FDR(X_oot,y_oot,clf))
    print("Bagging SVC: linear",c,"Time:", end - start)

Bagging SVC: linear 1 Time: 5631.959571838379
Bagging SVC: linear 10 Time: 89329.83380031586


In [27]:
kernal.append("linear")

In [28]:
# rbf variable combination
Cs = [0.01, 1, 10]
gammas = ["auto","scale"]
rbf_C_gammas = []
for i in Cs:
    for j in gammas:
        rbf_C_gammas.append((i,j))

In [30]:
# rbf 
for p in rbf_C_gammas:
    n_estimators = 10
    start = time.time()
    clf = BaggingClassifier(svm.SVC(kernel='rbf', probability=True,C=p[0],gamma=p[1]), max_samples=1.0 / n_estimators, n_estimators=n_estimators)
    clf.fit(x_train, y_train)
    end = time.time()
    kernal.append("rbf")
    C.append(p[0])
    gamma.append(p[1])
    train.append(FDR(x_train,y_train,clf))
    test.append(FDR(x_test,y_test,clf))
    ott.append(FDR(X_oot,y_oot,clf))
    print("Bagging SVC",c, end - start)

Bagging SVC 10 682.9935600757599
Bagging SVC 10 818.884920835495
Bagging SVC 10 611.2955009937286
Bagging SVC 10 630.9531073570251
Bagging SVC 10 765.9556128978729
Bagging SVC 10 810.4129018783569


In [ ]:
# poly
for c in Cs:
    n_estimators = 10
    start = time.time()
    clf = BaggingClassifier(svm.SVC(kernel='poly', probability=True,C=p[0],gamma=p[1]), max_samples=1.0 / n_estimators, n_estimators=n_estimators)
    clf.fit(x_train, y_train)
    end = time.time()
    kernal.append("poly")
    C.append(c)
    gamma.append("-")
    train.append(FDR(x_train,y_train,clf))
    test.append(FDR(x_test,y_test,clf))
    ott.append(FDR(X_oot,y_oot,clf))
    print("Bagging SVC",c, end - start)

In [32]:
C

[0.1, 1, 10, 0.01, 0.01, 1, 1, 10, 10]

In [33]:
# kernal = ["linear","linear","linear","rbf","rbf","rbf","rbf","rbf","rbf","poly","poly","poly"]

result = pd.DataFrame()
result["kernal"] = ["linear","linear","linear","rbf","rbf","rbf","rbf","rbf","rbf","poly","poly","poly"]
result["C"] = C
result["gamma"] = gamma
result["train"] = train
result["test"] = test
result["ott"] = ott

In [34]:
result

,kernal,C,gamma,train,test,ott
0,linear,0.10,-,0.504475,0.501876,0.487008
1,linear,1.00,-,0.506973,0.500625,0.487427
2,linear,10.00,-,0.504371,0.501042,0.483655
3,rbf,0.01,auto,0.526124,0.524385,0.501257
4,rbf,0.01,scale,0.526332,0.523968,0.502096
5,rbf,1.00,auto,0.520296,0.519800,0.504191
6,rbf,1.00,scale,0.521440,0.521050,0.505029
7,rbf,10.00,auto,0.513947,0.517299,0.503353
8,rbf,10.00,scale,0.515820,0.521050,0.505029


In [38]:
result.to_csv("svm_result.csv")

In [21]:
train_three_per = x_train1.shape[0]*0.003+1
test_three_per = x_test1.shape[0]*0.003+1
ott_three_per = ott.shape[0]*0.003+1

In [22]:
def fdr_calcualtion(model_fdr,score,y,fdr_list,three_per_slice,m):
    score.sort(reverse=True)
    model_fdr = model_fdr+sum(score[:three_per_slicea])/ int(y.sum())
    print (model_fdr)
    if m == 2:
        print(m)
        model_fdr = model_fdr/3
        fdr_list.append(model_fdr)

In [ ]:
# kernel = linear
# kernels=["linear","rbf","poly"]
Cs = [0.001, 0.01, 0.1, 1, 10]

train_fdr=[]
test_fdr= []
OTT_fdr = []
for c in Cs:
    model_fdr = 0
    for m in range(len(model)):
        svm_linear = svm.SVC(kernel='linear',C=c)
        svm_linear.fit(model[m][0], model[m][2])
        
        # predict train data 
        train_score = list(svm_linear.predict(model[m][0]))
        fdr_calcualtion(model_fdr,train_score,model[m][2],train_fdr,train_three_per,m)
        
        # predict test data 
        test_score = list(svm_linear.predict(model[m][1]))
        fdr_calcualtion(model_fdr,train_score,model[m][3],train_fdr,test_three_per,m)

        
        # predict ott data 
        OTT_score = list(svm_linear.predict(ott))
        fdr_calcualtion(model_fdr,OTT_score,ott,flabel_ott,OTT_fdr,ott_three_per,m)





In [ ]:
# kernal = poly

# kernel = linear
# kernels=["linear","rbf","poly"]

train_fdr=[]
test_fdr= []
OTT_fdr = []
for p in rbf_C_gammas:
    model_fdr = 0
    for m in range(len(model)):
        svm_linear = svm.SVC(kernel='poly',C=p[0],gammas=p[1])
        svm_linear.fit(model[m][0], model[m][2])
        
        # predict train data 
        train_score = list(svm_linear.predict(model[m][0]))
        fdr_calcualtion(model_fdr,train_score,model[m][2],train_fdr,train_three_per,m)
        
        # predict test data 
        test_score = list(svm_linear.predict(model[m][1]))
        fdr_calcualtion(model_fdr,train_score,model[m][3],train_fdr,test_three_per,m)

        
        # predict ott data 
        OTT_score = list(svm_linear.predict(ott))
        fdr_calcualtion(model_fdr,OTT_score,ott,flabel_ott,OTT_fdr,ott_three_per,m)





In [ ]:
# kernal = rbf



train_fdr=[]
test_fdr= []
OTT_fdr = []
for p in rbf_C_gammas:
    model_fdr = 0
    for m in range(len(model)):
        svm_linear = svm.SVC(kernel='rbf',C=p[0],gammas=p[1])
        svm_linear.fit(model[m][0], model[m][2])
        
        # predict train data 
        train_score = list(svm_linear.predict(model[m][0]))
        fdr_calcualtion(model_fdr,train_score,model[m][2],train_fdr,train_three_per,m)
        
        # predict test data 
        test_score = list(svm_linear.predict(model[m][1]))
        fdr_calcualtion(model_fdr,train_score,model[m][3],train_fdr,test_three_per,m)

        
        # predict ott data 
        OTT_score = list(svm_linear.predict(ott))
        fdr_calcualtion(model_fdr,OTT_score,ott,flabel_ott,OTT_fdr,ott_three_per,m)



